In [1]:
# Tugas Besar IF3270 Pembelajaran Mesin
# Anggota Kelompok:
# 1. Gede Prasidha Bhawarnawa - 13520004
# 2. Shadiq Harwiz - 13520038
# 3. Dwi Kalam Amal Tauhid - 13519210
# 4. Rahmat Rafid Akbar - 13520090

# Section Tugas Besar A: Implementasi Forward Propagation untuk Feed Forward Neural Network (FFNN)

# Import necessary libraries
import numpy as np

In [2]:
# Softmax Activation Function Implementation
def softmax(arr):
    numerator = np.exp(arr)
    denominator = np.sum(numerator)
    return numerator / denominator

In [3]:
# Sigmoid Activation Function Implementation
def sigmoid(arr):
    numerator = np.exp(arr)
    denominator = numerator + 1
    return numerator / denominator

In [4]:
# Linear Activation Function Implementation
def linear(arr):
    return arr

In [5]:
# Rectified Linear Unit (ReLU) Activation Function Implementation
def relu(arr):
    for i in range(len(arr)):
        if arr[i] < 0:
            arr[i] = 0
    return arr

In [6]:
# Format for the model input
# number_of_layers = INT
# number_of_nodes = INT
# loop (
# node_number, node_level, activation_function, weight
# )
# bias_1_weight,bias_2_weight,...,bias_n_weight

# By default, bias is 1 and automatically added to the model without the need to be specified in the text input
# The number of nodes does not include the bias node in each layer
# The number of bias weights is equal to the number of layers

# Example:
# 2
# 3
# node(1, 1, sigmoid, 0.1)
# node(2, 1, relu, 0.3)
# node(3, 2, linear, 0.5)
# 0.4,0.2

In [19]:
# Node Class and Instances
class Node:
    def __init__(self, node_number:int, node_level, activation_function, weight):
        self.node_number = node_number
        self.node_level = node_level
        self.activation_function = activation_function
        self.weight = weight
        self.output = 0
        self.error = 0
    
    def getNodeNumber(self):
        return self.node_number
    
    def getNodeLevel(self):
        return self.node_level
    
    def getActivationFunction(self):
        functionList = {
            1: sigmoid,
            2: relu,
            3: linear,
            4: softmax,
            5: bias
        }
        return functionList.get(self.activation_function, -1)
    
    def getWeight(self):
        return self.weight
    
    def getNodeInfo(self):
        print("Node Number: ", self.node_number)
        print("Node Level: ", self.node_level)
        print("Activation Function: ", self.activation_function)
        print("Weight: ", self.weight)
        print()

In [8]:
class Model:
    def __init__(self, number_of_layers:int, number_of_nodes:int, nodes:list):
        self.number_of_layers = number_of_layers
        self.number_of_nodes = number_of_nodes
        self.nodes = nodes # We assume that bias are also nodes of value 1
    
    def getNumberOfLayers(self):
        return self.number_of_layers
    
    def getNumberOfNodes(self):
        return self.number_of_nodes
    
    def getNodeList(self):
        return self.nodes
    
    def getNodeByIndex(self, index):
        return self.nodes[index]
    
    def getNodeByLevel(self, layer):
        nodes = []
        for node in self.nodes:
            if node.getNodeLevel() == layer:
                nodes.append(node)
        return nodes

In [22]:
# File reader and parser
from itertools import islice
import re
def readFile(filename):
    nodes =[]
    with open(filename, 'r') as f:
        contents = f.readlines()
        f.close()
    number_of_layers = int(contents[0])
    number_of_nodes = int(contents[1])
    for i in range(2, number_of_nodes + 2):
        node_info = contents[i].split(",")
        node = Node(int(node_info[0]), int(node_info[1]), node_info[2], float(node_info[3]))
        nodes.append(node)
    bias_weights = contents[number_of_nodes + 2].split(",")
    for layer in range(number_of_layers):
        number_of_nodes += 1
        node = Node(number_of_nodes, layer + 1, "bias", float(bias_weights[layer]))
        nodes.append(node)
    for node in nodes:
        node.getNodeInfo()
    
readFile('model1.txt')
    

Node Number:  1
Node Level:  1
Activation Function:  relu
Weight:  0.3

Node Number:  2
Node Level:  1
Activation Function:  sigmoid
Weight:  0.1

Node Number:  3
Node Level:  2
Activation Function:  linear
Weight:  0.5

Node Number:  4
Node Level:  2
Activation Function:  softmax
Weight:  0.7

Node Number:  5
Node Level:  2
Activation Function:  relu
Weight:  0.4

Node Number:  6
Node Level:  0
Activation Function:  bias
Weight:  0.2

Node Number:  7
Node Level:  1
Activation Function:  bias
Weight:  0.2

